# 🥱 專屬免費仔的 DSPy 上手指南

❤️ Created by [hengshiousheu](https://huggingface.co/Heng666).

本記事本讓你透過簡易的 QAChain 理解 [DSPy](https://github.com/stanfordnlp/dspy)如何讓你有序地編列語言模型管道。以下內容將提及到：

- 使用 vLLM 執行 AWQ 量化模型
- 實現自定義 BasicQA Module 來描述管道
- 使用 `DSPy.teleprompt` 編譯與最佳化模型
- 真實世界具備挑戰性資料集 BigBenchHard 進行 LLM 管道評估

*特別感謝 [Fearnworks 撰寫文章，有系統性整理 DSPy](https://fearnworks.github.io/blog/_pages/2024-02-12-DSPyForComplexSystemDevelopment.html), [Chris Levy 使用 Custom Dataset 來進行評估](https://drchrislevy.github.io/posts/dspy/dspy.html)*

以下內容我們將由 vLLM 配合 Mistral 模型進行一番練習，我們將透過一系列過程了解 DSPy 魅力所在

#建置環境

先來安裝所需要的套件環境

In [ ]:
!pip install --quiet dspy-ai vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.6/152.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

## 使用 vLLM 來執行

- 使用 `stdout` 將 log 進行輸出
- 選擇 AWQ 量化方式進行

In [ ]:
# Run server in the background
!nohup python -m vllm.entrypoints.openai.api_server --model TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ --quantization awq > server.log 2>&1 &

### 追蹤執行狀況

- 執行以下程式碼，來監測伺服器運作情況。
- 伺服器可能會花點時間來啟動
- 一但伺服器啟動之後，你會看到類似的 Log 資訊。
- ```INFO 02-17 09:20:39 api_server.py:209] args: Namespace(host=None, port=8000, allow_credentials=False, ```

In [ ]:
!tail server.log

model.safetensors: 100%|██████████| 4.15G/4.15G [00:44<00:00, 92.7MB/s]
INFO 02-17 10:13:03 llm_engine.py:337] # GPU blocks: 2120, # CPU blocks: 2048
INFO 02-17 10:13:07 model_runner.py:666] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 02-17 10:13:07 model_runner.py:670] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 02-17 10:13:23 model_runner.py:738] Graph capturing finished in 16 secs.
WARNING 02-17 10:13:23 serving_chat.py:264] No chat template provided. Chat API will not work.
INFO:     Started server process [1761]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0

In [ ]:
!curl http://localhost:8000/v1/models

{"object":"list","data":[{"id":"TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ","object":"model","created":1708164808,"owned_by":"vllm","root":"TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ","parent":null,"permission":[{"id":"modelperm-7e5384a6f62e414db5290a65ad5ac635","object":"model_permission","created":1708164808,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

#回家作業：逐漸建立複雜管道

- 實現一堆簽名和模組
- 使用多個提詞器來編譯和最佳化提示管道
- 使用 kNN Few Shot 和 Chain of Thought 作為解決方案的一部分非常重要
- 最後進行消融研究，透過 matplotlib 繪圖顯示各種參數和模組的重要性
- 使用 dspy 中的斷言和建議來進一步改進您的 dspy 程式、文件改進

* 無需使用 RAG 來完成此任務。

## 步驟一、設定語言模型

首先，我們將 DSPy 配置為使用針對指令最佳化的特定版本的 GPT 模型，確保我們的語言模型為詳細的指令追蹤任務做好準備。

In [ ]:
import dspy

# Set up the LM with GPT-3.5-turbo-instruct for detailed instruction following
# vLLM = dspy.HFClientVLLM(model="chienweichang/Breeze-7B-Instruct-v0_1-AWQ", port=8000, url="http://localhost", max_tokens=250)
vLLM = dspy.HFClientVLLM(model="TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ", port=8000, url="http://localhost")
dspy.settings.configure(lm=vLLM)

## 步驟二、載入資料集



BIG-Bench Hard 資料集（Suzgun 等人，2022）中有各種任務。當使用 load_dataset 載入該任務的相應記錄時，您可以使用這些字串之一。

```
['tracking_shuffled_objects_seven_objects', 'salient_translation_error_detection', 'tracking_shuffled_objects_three_objects', 'geometric_shapes', 'object_counting', 'word_sorting', 'logical_deduction_five_objects', 'hyperbaton', 'sports_understanding', 'logical_deduction_seven_objects', 'multistep_arithmetic_two', 'ruin_names', 'causal_judgement', 'logical_deduction_three_objects', 'formal_fallacies', 'snarks', 'boolean_expressions', 'reasoning_about_colored_objects', 'dyck_languages', 'navigate', 'disambiguation_qa', 'temporal_sequences', 'web_of_lies', 'tracking_shuffled_objects_five_objects', 'penguins_in_a_table', 'movie_recommendation', 'date_understanding']
```

In [ ]:
from datasets import load_dataset

ds = load_dataset("maveriq/bigbenchhard", "salient_translation_error_detection")["train"]
examples = [dspy.Example({"question": r["input"], "answer": r["target"]}).with_inputs("question") for r in ds]
print(f"There are {len(examples)} examples.")
trainset = examples[0:20]
valset = examples[20:]

Generating train split: 0 examples [00:00, ? examples/s]

There are 250 examples.


偷看一下題目大概長怎樣

In [ ]:
example = trainset[10]
for k, v in example.items():
    print(f"\n{k.upper()}:\n")
    print(v)


QUESTION:

The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names, places, locations, etc.) is changed to a different entity. Numerical Values: Numerical values (ordinals or cardinals), dates, and/or units are changed. Modifiers or Adjectives: The modifiers and adjectives pertaining to a noun are changed. Negation or Antonyms: Introduce or remove a negation or change comparatives to their antonyms. Facts: Trivial factual errors not pertaining to the above classes are introduced in the translations. Dropped Content: A significant clause in the translation is removed. Please identify that error.  Source: Ruth Lynn Deech, Baroness Deech DBE ist eine britische Juristin und Hochschullehrerin, die seit 2005 als Life Peeress Mitglied des House of Lords ist.
Translation: Ruth Lynn Deech, Baroness Deech DBE is a British lawyer and university lecturer who has been a life peer of the House of L

## 步驟三、定義 DSPy 模組

這是 DSPy 中的一個簡單模組，用於基本問答。

In [ ]:
class BasicQA(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.Predict("question -> answer")

    def forward(self, question):
        """forward 方法呼叫 __call__ ，類似於 pytorch 中的工作方式。"""
        return self.prog(question=question)


basic_qa = BasicQA()

將剛剛定義的 basic_qa 拿來做操作

In [ ]:
pred = basic_qa(question=example.question)
print("\nQUESTION:\n")
print(example.question)
print("\nANSWER:\n")
print(example.answer)
print("\nPREDICTION:\n")
print(pred.answer)


QUESTION:

The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names, places, locations, etc.) is changed to a different entity. Numerical Values: Numerical values (ordinals or cardinals), dates, and/or units are changed. Modifiers or Adjectives: The modifiers and adjectives pertaining to a noun are changed. Negation or Antonyms: Introduce or remove a negation or change comparatives to their antonyms. Facts: Trivial factual errors not pertaining to the above classes are introduced in the translations. Dropped Content: A significant clause in the translation is removed. Please identify that error.  Source: Ruth Lynn Deech, Baroness Deech DBE ist eine britische Juristin und Hochschullehrerin, die seit 2005 als Life Peeress Mitglied des House of Lords ist.
Translation: Ruth Lynn Deech, Baroness Deech DBE is a British lawyer and university lecturer who has been a life peer of the House of L

## 步驟四、定義評估指標

我們的評估指標將檢查 llm 輸出是否包含正確的多項選擇答案。為了在 DSPy 中定義評估指標，我們建立一個如下例所示的函數。前兩個輸入應該是 dspy.Example 的實例。度量函數可以包含評估任務所需的任何邏輯。您可以在文件中閱讀有關 trace 參數的更多資訊。它需要存在，即使沒有明確使用它。

In [ ]:
import re

def eval_metric(true, prediction, trace=None):
    pred = prediction.answer
    matches = re.findall(r"\([A-Z]\)", pred)
    parsed_answer = matches[-1] if matches else ""
    return parsed_answer == true.answer

我們建立了一個評估管道：

In [ ]:
from dspy.evaluate import Evaluate

evaluate = Evaluate(devset=valset, metric=eval_metric, num_threads=6, display_progress=True, display_table=10)

## 步驟五、執行第一次的評估

In [ ]:
evaluate(basic_qa)

Average Metric: 72 / 230  (31.3): 100%|██████████| 230/230 [01:40<00:00,  2.28it/s]

Average Metric: 72 / 230  (31.3%)



/usr/local/lib/python3.10/dist-packages/dspy/evaluate/evaluate.py:142: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


,question,example_answer,pred_answer,eval_metric
0,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(A),(D) Named Entities,False
1,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(B),(B) Numerical Values,✔️ [True]
2,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(F),(D) Named Entities,False
3,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),(D) Named Entities,✔️ [True]
4,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(A),(D) Named Entities,False
5,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),(D) Named Entities,✔️ [True]
6,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),(E) Dropped Content,False
7,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(F),(E) Dropped Content,False
8,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),(D) Named Entities,✔️ [True]
9,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),(E) Dropped Content,False


31.3

恭喜你成功達到第一次的完整管道練習

In [ ]:
vLLM.inspect_history(n=1)

同時可以取得 這次管道中所使用的 Prompt

## 步驟六、使用 DSPy 的 Optimizer(Formly Teleprompt) 進行Prompt最佳化

我們使用 DSPy 的 BootstrapFewShotWithRandomSearch 優化器來完善我們的 CoT 模組。此優化器會自動產生範例並迭代地提高模型在資料集上的效能。









選擇哪一種 Optimizer，[官方指南](https://dspy-docs.vercel.app/docs/building-blocks/optimizers#which-optimizer-should-i-use)說啥都不知道就選 ```BootstrapFewShotWithRandomSearch```

但有提供指南做選擇：
- 如果您的數據很少，例如您的任務的 10 個範例，請使用 ```BootstrapFewShot``` 。
- 如果您有更多的數據，例如50 個任務範例，請使用 ```BootstrapFewShotWithRandomSearch``` 。
- 如果您有更多數據，例如300 個或更多範例，請使用 ```BayesianSignatureOptimizer``` 。
- 如果您已經能夠將其中一個與大型 LM（例如，7B 參數或以上）一起使用並且需要一個非常高效的程序，請使用 ```BootstrapFinetune``` 將其編譯為小型 LM。

In [ ]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

config = dict(max_bootstrapped_demos=2, max_labeled_demos=4, num_candidate_programs=2, num_threads=6)

teleprompter = BootstrapFewShotWithRandomSearch(metric=eval_metric, **config)
optimized_qa = teleprompter.compile(basic_qa, trainset=trainset, valset=valset)

Going to sample between 1 and 2 traces per predictor.
Will attempt to train 2 candidate sets.


Average Metric: 72 / 230  (31.3): 100%|██████████| 230/230 [00:00<00:00, 276.80it/s]
/usr/local/lib/python3.10/dist-packages/dspy/evaluate/evaluate.py:142: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


Average Metric: 72 / 230  (31.3%)
Score: 31.3 for set: [0]
New best score: 31.3 for seed -3
Scores so far: [31.3]
Best score: 31.3


Average Metric: 102 / 230  (44.3): 100%|██████████| 230/230 [10:05<00:00,  2.63s/it]


Average Metric: 102 / 230  (44.3%)
Score: 44.35 for set: [4]
New best score: 44.35 for seed -2
Scores so far: [31.3, 44.35]
Best score: 44.35


...

Average Metric: 97 / 230  (42.2%)
Score: 42.17 for set: [4]
Scores so far: [31.3, 44.35, 47.83, 42.17]
Best score: 47.83
Average of max per entry across top 1 scores: 0.4782608695652174
Average of max per entry across top 2 scores: 0.691304347826087
Average of max per entry across top 3 scores: 0.7869565217391304
Average of max per entry across top 5 scores: 0.8478260869565217
Average of max per entry across top 8 scores: 0.8478260869565217
Average of max per entry across top 9999 scores: 0.8478260869565217


 20%|██        | 4/20 [00:29<01:58,  7.38s/it]


Bootstrapped 1 full traces after 5 examples in round 0.


Average Metric: 106 / 230  (46.1): 100%|██████████| 230/230 [11:35<00:00,  3.02s/it]

Average Metric: 106 / 230  (46.1%)
Score: 46.09 for set: [4]
Scores so far: [31.3, 44.35, 47.83, 42.17, 46.09]
Best score: 47.83
Average of max per entry across top 1 scores: 0.4782608695652174
Average of max per entry across top 2 scores: 0.7
Average of max per entry across top 3 scores: 0.8130434782608695
Average of max per entry across top 5 scores: 0.908695652173913
Average of max per entry across top 8 scores: 0.908695652173913
Average of max per entry across top 9999 scores: 0.908695652173913
5 candidate programs found.


(我花了 46 分鐘)
恭喜你終於優化完目前的模型哩，現在您可以評估優化後的模型，看看準確性是否有所提高。

In [ ]:
evaluate(optimized_qa)

Average Metric: 110 / 230  (47.8): 100%|██████████| 230/230 [00:00<00:00, 311.52it/s]


Average Metric: 110 / 230  (47.8%)


,question,example_answer,pred_answer,eval_metric
0,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(A),(C) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,False
1,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(B),(B) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,✔️ [True]
2,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(F),(F) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,✔️ [True]
3,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),(D) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,✔️ [True]
4,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(A),(E) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,False
5,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),(D) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,✔️ [True]
6,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),(D) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,✔️ [True]
7,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(F),(C) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,False
8,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),(D) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,✔️ [True]
9,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),(F) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,False


47.83

來看看得到良好的 Prompt

In [ ]:
vLLM.inspect_history(n=1)





Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Answer: ${answer}

---

Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names, places, locations, etc.) is changed to a different entity. Numerical Values: Numerical values (ordinals or cardinals), dates, and/or units are changed. Modifiers or Adjectives: The modifiers and adjectives pertaining to a noun are changed. Negation or Antonyms: Introduce or remove a negation or change comparatives to their antonyms. Facts: Trivial factual errors not pertaining to the above classes are introduced in the translations. Dropped Content: A significant clause in the translation is removed. Please identify that error. Source: Georg Otto Bussenius war ein deutscher Lehrer. Translation: Otto was a German teacher. The translation contains an error pertaining to Options: (

## 步驟七、比較兩者差異

此評估步驟使我們能夠將優化的 basic_qa 模組的性能與未優化的版本進行比較，從而為優化過程的有效性提供有價值的見解。結果？

In [ ]:
compiled_qa_score = evaluate(optimized_qa)
uncompiled_qa_score = evaluate(basic_qa)

print(f"## Retrieval Score for uncompiled : {uncompiled_qa_score}")
print(f"## Retrieval Score for compiled : {compiled_qa_score}")
print(f"Compilation Improvement: {compiled_qa_score - uncompiled_qa_score}%")

Average Metric: 110 / 230  (47.8): 100%|██████████| 230/230 [00:00<00:00, 316.72it/s]


Average Metric: 110 / 230  (47.8%)


,question,example_answer,pred_answer,eval_metric
0,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(A),(C) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,False
1,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(B),(B) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,✔️ [True]
2,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(F),(F) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,✔️ [True]
3,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),(D) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,✔️ [True]
4,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(A),(E) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,False
5,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),(D) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,✔️ [True]
6,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),(D) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,✔️ [True]
7,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(F),(C) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,False
8,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),(D) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,✔️ [True]
9,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),(F) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,False


Average Metric: 72 / 230  (31.3): 100%|██████████| 230/230 [00:00<00:00, 349.97it/s]


Average Metric: 72 / 230  (31.3%)


,question,example_answer,pred_answer,eval_metric
0,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(A),(D) Named Entities,False
1,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(B),(B) Numerical Values,✔️ [True]
2,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(F),(D) Named Entities,False
3,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),(D) Named Entities,✔️ [True]
4,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(A),(D) Named Entities,False
5,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),(D) Named Entities,✔️ [True]
6,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),(E) Dropped Content,False
7,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(F),(E) Dropped Content,False
8,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),(D) Named Entities,✔️ [True]
9,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),(E) Dropped Content,False


## Retrieval Score for uncompiled : 31.3
## Retrieval Score for compiled : 47.83
Compilation Improvement: 16.529999999999998%


## (star)休息一下，總結目前小重點

- 編寫小模組/程式。
- 選擇優化器。
- 運行編譯/最佳化步驟。
- 進行評估。

你省去了逐一監控 Prompt 對整個系統的影響，省去了手工編織良好的提示語時間，而從一個基礎的 prompt 得到還不錯的結果。

這真的很優秀，讓我們嘗試將思考鏈(Chain Of Though)想法納進來

## 步驟八、建立 DSPy模組，使用 思想鏈(Chain Of THough)
思維鏈 (CoT) 方法涉及模型生成通往最終答案的中間步驟，模仿人類如何解決推理問題。

In [ ]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought("question -> answer")

    def forward(self, question):
        answer = self.prog(question=question)
        return answer

cot_qa = CoT()

## 步驟九、進行 COT 管道評估

In [ ]:
evaluate(cot_qa)

Average Metric: 118 / 230  (51.3): 100%|██████████| 230/230 [04:27<00:00,  1.16s/it]

Average Metric: 118 / 230  (51.3%)


,question,example_answer,rationale,pred_answer,eval_metric
0,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(A),"identify the error in the translation. We can see that the source text mentions ""Ortsteilen Mitte und Tiergarten"" which translates to ""districts of Mitte and...",(A) Modifiers or Adjectives,✔️ [True]
1,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(B),"identify the error in the translation. We can see that the year of publication is incorrect. The correct year is 1759, not 1659.",(B) Numerical Values,✔️ [True]
2,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(F),"identify the error in the translation. We see that the original German sentence is about a type of plant, specifically ""Kohl"" which means ""Cabbage"" in...",(D) Named Entities,False
3,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),"identify the error in the translation. We can see that the source text and the translation are quite similar, but there is a slight difference...",(A) Modifiers or Adjectives,False
4,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(A),"identify the error in the translation. We see that the source sentence and the translation are almost identical. The only difference is the word ""ehemalige""...",(A) Modifiers or Adjectives,✔️ [True]
5,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),"identify the error in the translation. We can see that the city name ""Sassnitz"" is changed to ""Seibnitz"" in the translation. This is an error...",(D) Named Entities,✔️ [True]
6,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),identify the error in the translation. We can see that the location of Elbenberg is incorrect in the translation. The correct location is that it...,(D) Named Entities,✔️ [True]
7,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(F),"identify the error in the translation. We can see that the source text mentions ""Chūō ist einer der 23 Bezirke im Osten der japanischen Präfektur...",(D) Named Entities,False
8,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),"identify the error in the translation. We can see that the source text mentions ""Unter dem Namen Espenlaub"" which is changed to ""Under the name...",(D) Named Entities,✔️ [True]
9,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),"identify the error in the translation. We can see that the source text mentions the ""Haager Konvention"" (Hague Convention), but the translation incorrectly refers to...",(D) Named Entities,✔️ [True]


51.3

來檢視一下，COT 的 Prompt

In [ ]:
vLLM.inspect_history(n=1)





Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names, places, locations, etc.) is changed to a different entity. Numerical Values: Numerical values (ordinals or cardinals), dates, and/or units are changed. Modifiers or Adjectives: The modifiers and adjectives pertaining to a noun are changed. Negation or Antonyms: Introduce or remove a negation or change comparatives to their antonyms. Facts: Trivial factual errors not pertaining to the above classes are introduced in the translations. Dropped Content: A significant clause in the translation is removed. Please identify that error. Source: Auf dieser Seite sind die Baudenkmäler der schwäbischen Gemeinde Ma

## 步驟十、使用 DSPy 的 Optimizer(Formly Teleprompt) 進行最佳化

我們使用 DSPy 的 BootstrapFewShotWithRandomSearch 優化器來完善我們的 CoT 模組。此優化器會自動產生範例並迭代地提高模型在資料集上的效能。

In [ ]:
config = dict(max_bootstrapped_demos=1, max_labeled_demos=4, num_candidate_programs=4, num_threads=6)

teleprompter = BootstrapFewShotWithRandomSearch(metric=eval_metric, **config)
optimized_cot_qa = teleprompter.compile(cot_qa, trainset=trainset, valset=valset)

Going to sample between 1 and 1 traces per predictor.
Will attempt to train 4 candidate sets.


Average Metric: 118 / 230  (51.3): 100%|██████████| 230/230 [00:00<00:00, 312.40it/s]


Average Metric: 118 / 230  (51.3%)
Score: 51.3 for set: [0]
New best score: 51.3 for seed -3
Scores so far: [51.3]
Best score: 51.3


Average Metric: 130 / 230  (56.5): 100%|██████████| 230/230 [09:36<00:00,  2.51s/it]


Average Metric: 130 / 230  (56.5%)
Score: 56.52 for set: [4]
New best score: 56.52 for seed -2
Scores so far: [51.3, 56.52]
Best score: 56.52


...

Average Metric: 107 / 230  (46.5%)
Score: 46.52 for set: [4]
Scores so far: [51.3, 56.52, 46.52, 52.61, 48.26, 46.52]
Best score: 56.52
Average of max per entry across top 1 scores: 0.5652173913043478
Average of max per entry across top 2 scores: 0.7956521739130434
Average of max per entry across top 3 scores: 0.9
Average of max per entry across top 5 scores: 0.9652173913043478
Average of max per entry across top 8 scores: 0.9695652173913043
Average of max per entry across top 9999 scores: 0.9695652173913043


  5%|▌         | 1/20 [00:07<02:15,  7.15s/it]


Bootstrapped 1 full traces after 2 examples in round 0.


Average Metric: 121 / 230  (52.6): 100%|██████████| 230/230 [08:19<00:00,  2.17s/it]

Average Metric: 121 / 230  (52.6%)
Score: 52.61 for set: [4]
Scores so far: [51.3, 56.52, 46.52, 52.61, 48.26, 46.52, 52.61]
Best score: 56.52
Average of max per entry across top 1 scores: 0.5652173913043478
Average of max per entry across top 2 scores: 0.7956521739130434
Average of max per entry across top 3 scores: 0.8913043478260869
Average of max per entry across top 5 scores: 0.9695652173913043
Average of max per entry across top 8 scores: 0.9826086956521739
Average of max per entry across top 9999 scores: 0.9826086956521739
7 candidate programs found.


(51 mins later)
這一步就是神奇發生的地方。優化器會自動調整 CoT 模組的提示和權重，以最大化所需的結果指標，例如準確性和連貫性。這種自動化顯著減少了手動微調和快速製作的需要，從而簡化了開發過程。

讓我們來比較看看差異

In [ ]:
compiled_cot_qa_score = evaluate(optimized_cot_qa)
uncompiled_cot_qa_score = evaluate(cot_qa)

print(f"## Retrieval Score for uncompiled : {uncompiled_cot_qa_score}")
print(f"## Retrieval Score for compiled : {compiled_cot_qa_score}")
print(f"Compilation Improvement: {compiled_cot_qa_score - uncompiled_cot_qa_score}%")

Average Metric: 130 / 230  (56.5): 100%|██████████| 230/230 [00:00<00:00, 326.53it/s]


Average Metric: 130 / 230  (56.5%)


,question,example_answer,rationale,pred_answer,eval_metric
0,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(A),"identify the error in the translation. We can see that the source text mentions ""placesartiger Verkehrsknotenpunkt"" which means ""square-like hub for traffic"". However, the translation...",(A) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,✔️ [True]
1,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(B),"identify the error in the translation. We notice that the year of publication is incorrect. The correct year is 1759, not 1659.",(B) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,✔️ [True]
2,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(F),"identify the error in the translation. We know that the source text is about a type of plant, specifically a member of the Cruciferous Family....",(F) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,✔️ [True]
3,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),"identify the error in the translation. We can see that the source text mentions ""Richard Raphael Roland Risse"" as the name of the painter, but...",(D) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,✔️ [True]
4,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(A),"identify the error in the translation. We can see that the source text mentions that Beth Clare Rodford is a ""former"" British rower. However, the...",(E) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,False
5,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),"identify the error in the translation. We notice that the name of the city has been changed from ""Sassnitz"" to ""Seibnitz"". This is a clear...",(D) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,✔️ [True]
6,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),identify the error in the translation. We can see that the location of Elbenberg is incorrect in the translation. The correct location is a district...,(D) --- Question: The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities:...,✔️ [True]
7,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(F),"identify the error in the translation. We can see that the source text mentions ""Chūō ist einer der 23 Bezirke im Osten der japanischen Präfektur...",(C) --- Question: The following translations,False
8,"The following translations from German to English contain a particular error. That error will be one 

Average Metric: 118 / 230  (51.3): 100%|██████████| 230/230 [00:00<00:00, 364.03it/s]


Average Metric: 118 / 230  (51.3%)


,question,example_answer,rationale,pred_answer,eval_metric
0,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(A),"identify the error in the translation. We can see that the source text mentions ""Ortsteilen Mitte und Tiergarten"" which translates to ""districts of Mitte and...",(A) Modifiers or Adjectives,✔️ [True]
1,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(B),"identify the error in the translation. We can see that the year of publication is incorrect. The correct year is 1759, not 1659.",(B) Numerical Values,✔️ [True]
2,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(F),"identify the error in the translation. We see that the original German sentence is about a type of plant, specifically ""Kohl"" which means ""Cabbage"" in...",(D) Named Entities,False
3,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),"identify the error in the translation. We can see that the source text and the translation are quite similar, but there is a slight difference...",(A) Modifiers or Adjectives,False
4,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(A),"identify the error in the translation. We see that the source sentence and the translation are almost identical. The only difference is the word ""ehemalige""...",(A) Modifiers or Adjectives,✔️ [True]
5,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),"identify the error in the translation. We can see that the city name ""Sassnitz"" is changed to ""Seibnitz"" in the translation. This is an error...",(D) Named Entities,✔️ [True]
6,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),identify the error in the translation. We can see that the location of Elbenberg is incorrect in the translation. The correct location is that it...,(D) Named Entities,✔️ [True]
7,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(F),"identify the error in the translation. We can see that the source text mentions ""Chūō ist einer der 23 Bezirke im Osten der japanischen Präfektur...",(D) Named Entities,False
8,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),"identify the error in the translation. We can see that the source text mentions ""Unter dem Namen Espenlaub"" which is changed to ""Under the name...",(D) Named Entities,✔️ [True]
9,"The following translations from German to English contain a particular error. That error will be one of the following types: Named Entities: An entity (names,...",(D),"identify the error in the translation. We can see that the source text mentions the ""Haager Konvention"" (Hague Convention), but the translation incorrectly refers to...",(D) Named Entities,✔️ [True]


## Retrieval Score for uncompiled : 51.3
## Retrieval Score for compiled : 56.52
Compilation Improvement: 5.220000000000006%


## 步驟十ㄧ、保存模型

做了這麼多訓練跟驗證，總該保存吧～

In [ ]:
optimized_qa.save(path="optimized_qa_module.json")

In [ ]:
optimized_cot_qa.save(path="optimized_cot_qa_module.json")

此評估步驟使我們能夠將優化的 CoT 模組的性能與未優化的版本進行比較，從而為優化過程的有效性提供有價值的見解。結果？

## 小結重點整理

DSPy 不僅簡化了目前使用 LM 開發複雜系統的流程，而且還為創新開闢了新的可能性。透過減少對手動提示製作和微調的依賴，開發人員可以更專注於其專案的創意和策略方面。這種轉變有可能加速更複雜、更通用的系統的開發，突破語言模型的可能性界線。

DSPy 代表了語言模型複雜系統開發領域的重大飛躍。它能夠自動化和優化這些系統中語言模型的使用，不僅可以簡化開發過程，還可以增強所創建解決方案的穩健性和可擴展性。展望未來，我們很清楚像 DSPy 這樣的框架和技術將發揮關鍵作用。